# Morris Screening

In this notebook we apply the popular Morris screening method to a building design problem.  
We determine the sensitivty of the objective (electricty use) to each of the design parameters.

In [1]:
import numpy as np
import pandas as pd

from SALib.sample import morris as msampling
from SALib.analyze import morris as manalysis

from besos import eppy_funcs as ef 
import besos.sampling as sampling        
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist
from parameter_sets import parameter_set

## Build an EnergyPlus Evaluator

In [2]:
parameters = parameter_set(7) # use a pre-defined parameter set
problem = EPProblem(parameters, ['Electricity:Facility'])
building = ef.get_building() # use the example building
evaluator = EvaluatorEP(problem, building)
inputs = sampling.dist_sampler(sampling.lhs, problem, 50) # get 50 samples of the input space

/home/evan/anaconda3/lib/python3.6/site-packages/besos/problem.py:77: RuntimeWarning: Duplicate names found. (duplicate, repetitions): [('Watts per Zone Floor Area', 2)]
Attempting to fix automatically
  warnings.warn(RuntimeWarning(f'Duplicate names found. (duplicate, repetitions): '


## Conduct a Morris screening of the parameters

The following cells conduct a Morris screening, a global sensitivity method.  
It uses $r$ times $n$ one-at-time changes (OAT) of each parameter at randomly selected points.  
The resulting distribution of $r$ samples provides a mean $\mu^*$, and a standard deviation $\sigma$ of the elementary effects of the $i$-th input parameter. [[1]] [[2]] [[3]]

[1]: https://www.sciencedirect.com/science/article/pii/S1364032112007101
[2]: https://en.wikipedia.org/wiki/Elementary_effects_method
[3]: https://en.wikipedia.org/wiki/Morris_method

In [3]:
names =[parameters[i].name for i in range(len(parameters))]
bounds=[[parameters[i].value_descriptor.min, parameters[i].value_descriptor.max] for i in range(len(parameters))]

problem = {
    'num_vars': len(parameters),
    'names': names,
    'bounds': bounds
                    }

X = np.round(msampling.sample(problem, N=5), decimals=3)
inputs = pd.DataFrame(data=X,columns=names)
outputs = evaluator.df_apply(inputs)


Y=outputs.values
Si = manalysis.analyze(problem, X, Y, conf_level=0.95,print_to_console=True, num_levels=4)
pd.DataFrame(data=Si['mu_star'], index=Si['names']).sort_values(by=0)


Parameter                         Mu_Star         Mu    Mu_Star_Conf      Sigma
Wall conductivity              51681025.010 51681025.010    14679736.172 18632485.565
Attic thickness                4785764.450 2982208.365     3961702.784 6492376.442
U-Factor                       3683527.564 -3683527.564     1252366.499 1564069.477
Solar Heat Gain Coefficient    56489897.949 56489897.949     7139628.966 9485776.437
Watts per Zone Floor Area_0    320661019.660 320661019.660     1558388.071 1963860.854
Watts per Zone Floor Area_1    316143069.669 316143069.669     2219941.236 2891674.839
Window to Wall Ratio                0.000      0.000           0.000      0.000


,0
Window to Wall Ratio,0.000000e+00
U-Factor,3.683528e+06
Attic thickness,4.785764e+06
Wall conductivity,5.168103e+07
Solar Heat Gain Coefficient,5.648990e+07
Watts per Zone Floor Area_1,3.161431e+08
Watts per Zone Floor Area_0,3.206610e+08
